In [1]:
import os

import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sensor_data_path = "./sensor_data"

encoder_data = pd.read_csv(os.path.join(sensor_data_path,"encoder.csv"), names=["timestamp", "left_count", "right_count"])
fog_data = pd.read_csv(os.path.join(sensor_data_path,"fog.csv"), names=["timestamp", "delta_roll", "delta_pitch", "delta_yaw"])


# header names for lidar_data
lidar_angle = np.around(np.linspace(-5, 185, 286,),1).astype('str').tolist()
lidar_angle.insert(0, "timestamp")

lidar_data = pd.read_csv(os.path.join(sensor_data_path,"lidar.csv"),names=lidar_angle)

In [3]:
# first column represents the timestamp of the observation
'''
FOV: 190 (degree)
Start angle: -5 (degree)
End angle: 185 (degree)
Angular resolution: 0.666 (degree) # shoot a laser ray every 0.666 degree
Max range: 80 (meter)

* LiDAR rays with value 0.0 represent infinite range observations.
'''
print(lidar_data.shape)
lidar_data.head(5)
# elment in 1:end columns is the range (distance in meter)

(115865, 287)


,timestamp,-5.0,-4.3,-3.7,-3.0,-2.3,-1.7,-1.0,-0.3,0.3,1.0,1.7,2.3,3.0,3.7,4.3,5.0,5.7,6.3,7.0,7.7,8.3,9.0,9.7,10.3,11.0,11.7,12.3,13.0,13.7,14.3,15.0,15.7,16.3,17.0,17.7,18.3,19.0,19.7,20.3,21.0,21.7,22.3,23.0,23.7,24.3,25.0,25.7,26.3,27.0,27.7,28.3,29.0,29.7,30.3,31.0,31.7,32.3,33.0,33.7,34.3,35.0,35.7,36.3,37.0,37.7,38.3,39.0,39.7,40.3,41.0,41.7,42.3,43.0,43.7,44.3,45.0,45.7,46.3,47.0,47.7,48.3,49.0,49.7,50.3,51.0,51.7,52.3,53.0,53.7,54.3,55.0,55.7,56.3,57.0,57.7,58.3,59.0,59.7,60.3,61.0,61.7,62.3,63.0,63.7,64.3,65.0,65.7,66.3,67.0,67.7,68.3,69.0,69.7,70.3,71.0,71.7,72.3,73.0,73.7,74.3,75.0,75.7,76.3,77.0,77.7,78.3,79.0,79.7,80.3,81.0,81.7,82.3,83.0,83.7,84.3,85.0,85.7,86.3,87.0,87.7,88.3,89.0,89.7,90.3,91.0,91.7,92.3,93.0,93.7,94.3,95.0,95.7,96.3,97.0,97.7,98.3,99.0,99.7,100.3,101.0,101.7,102.3,103.0,103.7,104.3,105.0,105.7,106.3,107.0,107.7,108.3,109.0,109.7,110.3,111.0,111.7,112.3,113.0,113.7,114.3,115.0,115.7,116.3,117.0,117.7,118.3,119.0,119.7,120.3,121.0,121.7,122.3,123.0,123.7,124.3,125.0,125.7,126.3,127.0,127.7,128.3,129.0,129.7,130.3,131.0,131.7,132.3,133.0,133.7,134.3,135.0,135.7,136.3,137.0,137.7,138.3,139.0,139.7,140.3,141.0,141.7,142.3,143.0,143.7,144.3,145.0,145.7,146.3,147.0,147.7,148.3,149.0,149.7,150.3,151.0,151.7,152.3,153.0,153.7,154.3,155.0,155.7,156.3,157.0,157.7,158.3,159.0,159.7,160.3,161.0,161.7,162.3,163.0,163.7,164.3,165.0,165.7,166.3,167.0,167.7,168.3,169.0,169.7,170.3,171.0,171.7,172.3,173.0,173.7,174.3,175.0,175.7,176.3,177.0,177.7,178.3,179.0,179.7,180.3,181.0,181.7,182.3,183.0,183.7,184.3,185.0
0,1544582648750747670,30.390001,30.648001,32.206001,32.182003,32.926003,33.361000,37.939003,33.078003,37.734001,38.010002,38.247002,31.712002,39.507000,30.747002,30.648001,25.324001,0.000000,35.225002,9.619000,9.915001,25.653002,9.640000,9.661000,25.401001,24.107000,23.971001,28.421001,0.000000,0.000000,36.106003,41.432003,0.000000,0.0,0.0,0.000000,26.910002,0.0,0.0,0.0,0.0,0.0,0.000000,10.731001,10.423000,10.261001,10.521001,38.759003,0.000000,0.0,0.0,0.0,0.0,60.240002,0.0,0.0,0.000000,56.602001,56.502003,55.628002,0.000000,53.644001,53.936001,54.420002,0.000000,55.447002,55.987003,56.538002,57.133003,57.710003,58.348003,58.975002,59.635002,60.342003,61.046001,61.780003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,9.064000,9.102000,9.114000,6.189,6.187,6.203,6.153,5.003,4.985000,5.089000,4.469000,5.272,5.267,4.330,4.347,18.443001,4.322,4.279,4.355,4.395,4.442,4.470,4.541,4.572,4.648000,5.286000,5.345000,10.781000,4.620000,4.659,4.606,3.842000,3.907,4.632,4.175,4.156,4.977,4.171000,5.083,3.970,3.975,3.917,3.530,4.901,4.878,3.967,3.990,5.203,6.176,6.307000,5.155000,5.118000,3.739,8.384001,3.824000,3.951000,3.803,3.733000,3.751,3.799,10.494000,10.594001,10.509001,10.915001,10.094001,10.150001,11.482000,10.349001,10.089001,9.183001,9.027000,11.756001,13.469001,0.000000,7.940001,7.954000,7.944000,7.953001,8.049001,8.024000,8.011001,7.990000,8.217000,8.452001,8.237000,8.105000,8.370001,8.853001,11.180000,8.305000,8.649,11.253000,9.644000,14.464001,14.450001,12.367001,11.971001,11.640000,11.322001,11.033001,10.827001,10.637000,10.383000,10.244000,10.104000,9.977000,9.851001,9.741000,9.593000
1,1544582648760541220,30.431002,30.645002,32.213001,32.200001,32.913002,33.374001,37.942001,33.096001,37.723003,38.029003,38.308002,31.684002,39.521004,30.682001,30.615002,25.349001,38.952003,9.614000,9.815001,26.299002,25.645000,9.619000,9.842000,25.367001,24.121000,23.978001,0.000000,29.442001,0.000000,0.000000,41.556004,0.000000,0.0,0.0,0.000000,26.881001,0.0,0.0,0.0,0.0,0.0,0.000000,10.734000,10.463000,10.354000,10.438001,38.730003,38.923000,0.0,0.0,

In [4]:
# Only consider points between the minimum and maximum range of the sensor.
# Measurements between 2m-75m are recommended to be included as valid data.


In [5]:
'''
lidar_data = lidar_data.drop(["timestamp"], axis = 1)
lidar_data.head()
lidar_data.plot.hist(bins=10)
plt.show()
'''

'\nlidar_data = lidar_data.drop(["timestamp"], axis = 1)\nlidar_data.head()\nlidar_data.plot.hist(bins=10)\nplt.show()\n'

In [6]:
##################################################################################################################

## Encoder

In [7]:
'''
Encoder calibrated parameter
Encoder resolution: 4096
Encoder left wheel diameter: 0.623479
Encoder right wheel diameter: 0.622806
Encoder wheel base: 1.52439

* The encoder data is stored as [timestamp, left count, right count].
'''
print(encoder_data.shape)
encoder_data.head(10)
#1544582648750 747670
#1544582648760 541220

(116048, 3)


,timestamp,left_count,right_count
0,1544582648751852447,69577,69699
1,1544582648761750060,69673,69795
2,1544582648771748778,69769,69891
3,1544582648781765037,69865,69986
4,1544582648791768265,69961,70081
5,1544582648801801135,70057,70178
6,1544582648811766755,70154,70273
7,1544582648821765891,70251,70369
8,1544582648831862776,70348,70467
9,1544582648841781008,70445,70565


In [8]:
df = encoder_data.copy()
print(df.shape)
df = np.diff(df, axis=0)
df=np.vstack(([0,0,0],df))
# df

(116048, 3)


In [9]:
new = {'dt': df[:,0]*1e-9, 'dlz': df[:,1], 'drz': df[:,2]}
new_df = pd.DataFrame(data=new)
new_df.head()

,dt,dlz,drz
0,0.000000,0,0
1,0.009898,96,96
2,0.009999,96,96
3,0.010016,96,95
4,0.010003,96,95


In [10]:
res = 4096
dL = 0.623479
dR = 0.622806
new_df["timestamp"] = encoder_data["timestamp"]
new_df["distance_left"] = np.pi * dL/res * new_df["dlz"]
new_df["distance_right"] = np.pi * dR/res * new_df["drz"]
new_df['velocity_left'] = new_df["distance_left"] /  new_df["dt"]
new_df['velocity_right'] = new_df["distance_right"] /  new_df["dt"]
new_df['linear_velocity(m/s)'] = (new_df['velocity_left'] + new_df['velocity_right'])/2.0 
# new_df['linear_velocity(mph)'] = new_df['linear_velocity(m/s)']*2.237
new_df.fillna(0, inplace=True)

new_df.head()

,dt,dlz,drz,timestamp,distance_left,distance_right,velocity_left,velocity_right,linear_velocity(m/s)
0,0.000000,0,0,1544582648751852447,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.009898,96,96,1544582648761750060,0.045907,0.045858,4.638233,4.633226,4.635729
2,0.009999,96,96,1544582648771748778,0.045907,0.045858,4.591332,4.586376,4.588854
3,0.010016,96,95,1544582648781765037,0.045907,0.045380,4.583291,4.530653,4.556972
4,0.010003,96,95,1544582648791768265,0.045907,0.045380,4.589262,4.536555,4.562908


In [11]:
# print(sum(new_df['distance_left']), sum(new_df['distance_right']))
# new_df.describe()

In [12]:
##################################################################################################################

## FOG 

In [13]:
'''
FOG (Fiber Optic Gyro) extrinsic calibration parameter from vehicle
RPY(roll/pitch/yaw = XYZ extrinsic, degree), R(rotation matrix), T(translation matrix, meter)
RPY: 0 0 0
R: 1 0 0 0 1 0 0 0 1 
T: -0.335 -0.035 0.78

* The sensor measurements are stored as [timestamp, delta roll, delta pitch, delta yaw] in radians.
'''

print(fog_data.shape)
# fog_data.head(20)


(1160508, 4)


In [14]:
fog_data['dt'] = np.hstack((0,np.diff(fog_data['timestamp']*1e-9)))

In [15]:
pd.set_option("precision", 10)

In [16]:
# 1544582648750 747670
# 1544582648760 541220
fog_data.head(10)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt
0,1544582648743131772,-0.0000050486,0.0000275229,-0.0000119692,0.0000000000
1,1544582648744114218,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230
2,1544582648745131427,0.0000019153,0.0000220933,-0.0000103648,0.0010175705
3,1544582648746103145,0.0000166523,0.0000216797,-0.0000098693,0.0009713173
4,1544582648747106429,0.0000310455,0.0000226423,-0.0000097198,0.0010032654
5,1544582648748126535,0.0000394436,0.0000223999,-0.0000096258,0.0010201931
6,1544582648749104662,0.0000353109,0.0000206480,-0.0000097710,0.0009782314
7,1544582648750131486,0.0000211805,0.0000173981,-0.0000095623,0.0010268688
8,1544582648751099508,0.0000026081,0.0000128376,-0.0000088801,0.0009679794
9,1544582648752131818,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524


# sychronize

In [17]:
lidar_df = lidar_data.copy()
fog_df = fog_data.copy()
encoder_df = new_df.copy()
encoder_df.drop(["dt","dlz", "drz", "distance_left", "distance_right","velocity_left","velocity_right"], axis=1, inplace=True)

In [18]:
fog_df.head()

,timestamp,delta_roll,delta_pitch,delta_yaw,dt
0,1544582648743131772,-0.0000050486,0.0000275229,-0.0000119692,0.0000000000
1,1544582648744114218,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230
2,1544582648745131427,0.0000019153,0.0000220933,-0.0000103648,0.0010175705
3,1544582648746103145,0.0000166523,0.0000216797,-0.0000098693,0.0009713173
4,1544582648747106429,0.0000310455,0.0000226423,-0.0000097198,0.0010032654


In [19]:
encoder_df.head()

,timestamp,linear_velocity(m/s)
0,1544582648751852447,0.0000000000
1,1544582648761750060,4.6357292318
2,1544582648771748778,4.5888536820
3,1544582648781765037,4.5569718994
4,1544582648791768265,4.5629081732


In [20]:
fog_df['timestamp']=np.round(fog_df['timestamp']*1e-6)

In [21]:
print(fog_df.shape)
fog_df.head(10)

(1160508, 5)


,timestamp,delta_roll,delta_pitch,delta_yaw,dt
0,1544582648743.0,-0.0000050486,0.0000275229,-0.0000119692,0.0000000000
1,1544582648744.0,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230
2,1544582648745.0,0.0000019153,0.0000220933,-0.0000103648,0.0010175705
3,1544582648746.0,0.0000166523,0.0000216797,-0.0000098693,0.0009713173
4,1544582648747.0,0.0000310455,0.0000226423,-0.0000097198,0.0010032654
5,1544582648748.0,0.0000394436,0.0000223999,-0.0000096258,0.0010201931
6,1544582648749.0,0.0000353109,0.0000206480,-0.0000097710,0.0009782314
7,1544582648750.0,0.0000211805,0.0000173981,-0.0000095623,0.0010268688
8,1544582648751.0,0.0000026081,0.0000128376,-0.0000088801,0.0009679794
9,1544582648752.0,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524


In [22]:
encoder_df['timestamp']=np.round(encoder_df['timestamp']*1e-6)-1

In [23]:
print(encoder_df.shape)
encoder_df.head(10)

(116048, 2)


,timestamp,linear_velocity(m/s)
0,1544582648751.0,0.0000000000
1,1544582648761.0,4.6357292318
2,1544582648771.0,4.5888536820
3,1544582648781.0,4.5569718994
4,1544582648791.0,4.5629081732
5,1544582648801.0,4.5970392340
6,1544582648811.0,4.6041201653
7,1544582648821.0,4.6125740375
8,1544582648831.0,4.6152294822
9,1544582648841.0,4.6983616970


In [24]:
mergedf_fog_encoder_inner =fog_df.merge(encoder_df, how="inner",)

In [25]:
mergedf_fog_encoder_left =fog_df.merge(encoder_df, how="left",)

In [26]:
print(mergedf_fog_encoder_inner.shape)
print(mergedf_fog_encoder_left.shape)

(116150, 6)
(1160508, 6)


In [27]:
mergedf_fog_encoder_inner.head(5)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
0,1544582648751.0,0.0000026081,0.0000128376,-0.0000088801,0.0009679794,0.0000000000
1,1544582648761.0,-0.0000192591,-0.0000029518,-0.0000091783,0.0010025501,4.6357292318
2,1544582648771.0,0.0000076882,-0.0000124863,-0.0000073896,0.0009934902,4.5888536820
3,1544582648781.0,0.0000201547,-0.0000002885,-0.0000076526,0.0009822845,4.5569718994
4,1544582648791.0,-0.0000140254,-0.0000076365,-0.0000098158,0.0010175705,4.5629081732


In [28]:
mergedf_fog_encoder_left.head(30)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
0,1544582648743.0,-0.0000050486,0.0000275229,-0.0000119692,0.0000000000,NaN
1,1544582648744.0,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230,NaN
2,1544582648745.0,0.0000019153,0.0000220933,-0.0000103648,0.0010175705,NaN
3,1544582648746.0,0.0000166523,0.0000216797,-0.0000098693,0.0009713173,NaN
4,1544582648747.0,0.0000310455,0.0000226423,-0.0000097198,0.0010032654,NaN
5,1544582648748.0,0.0000394436,0.0000223999,-0.0000096258,0.0010201931,NaN
6,1544582648749.0,0.0000353109,0.0000206480,-0.0000097710,0.0009782314,NaN
7,1544582648750.0,0.0000211805,0.0000173981,-0.0000095623,0.0010268688,NaN
8,1544582648751.0,0.0000026081,0.0000128376,-0.0000088801,0.0009679794,0.0000000000
9,1544582648752.0,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524,NaN


In [29]:
mergedf_fog_encoder_left.fillna(method='backfill',inplace=True)
mergedf_fog_encoder_left.head(50)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
0,1544582648743.0,-0.0000050486,0.0000275229,-0.0000119692,0.0000000000,0.0000000000
1,1544582648744.0,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230,0.0000000000
2,1544582648745.0,0.0000019153,0.0000220933,-0.0000103648,0.0010175705,0.0000000000
3,1544582648746.0,0.0000166523,0.0000216797,-0.0000098693,0.0009713173,0.0000000000
4,1544582648747.0,0.0000310455,0.0000226423,-0.0000097198,0.0010032654,0.0000000000
5,1544582648748.0,0.0000394436,0.0000223999,-0.0000096258,0.0010201931,0.0000000000
6,1544582648749.0,0.0000353109,0.0000206480,-0.0000097710,0.0009782314,0.0000000000
7,1544582648750.0,0.0000211805,0.0000173981,-0.0000095623,0.0010268688,0.0000000000
8,1544582648751.0,0.0000026081,0.0000128376,-0.0000088801,0.0009679794,0.0000000000
9,1544582648752.0,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524,4.6357292318


In [30]:
mergedf_fog_encoder_left.tail(20)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
1160488,1544583809212.0,0.0000012073,-0.0000025920,-0.0000007287,0.0009984970,6.8122031193
1160489,1544583809213.0,0.0000056566,0.0000038938,0.0000000542,0.0009829998,6.8122031193
1160490,1544583809214.0,0.0000067532,0.0000093901,0.0000002716,0.0010194778,6.8122031193
1160491,1544583809215.0,0.0000045995,0.0000124650,0.0000010562,0.0010013580,6.8122031193
1160492,1544583809216.0,-0.0000025343,0.0000116023,0.0000012374,0.0010187626,6.8122031193
1160493,1544583809217.0,-0.0000095242,0.0000078488,0.0000017970,0.0009527206,6.8122031193
1160494,1544583809218.0,-0.0000128561,0.0000040793,0.0000018875,0.0010383129,6.8103277505
1160495,1544583809219.0,-0.0000131552,0.0000022325,0.0000017695,0.0009620190,6.8103277505
1160496,1544583809220.0,-0.0000076909,0.0000031329,0.0000018200,0.0010004044,6.8103277505
1160497,1544583809221.0,-0.0000007763,0.0000060454,0.0000015833,0.0010015965,6.8103277505


In [31]:
mergedf_fog_encoder_left.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                      False
linear_velocity(m/s)     True
dtype: bool

In [32]:
mergedf_fog_encoder_left.fillna(method='ffill',inplace=True)
mergedf_fog_encoder_left.tail(10)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
1160498,1544583809222.0,0.0000055917,0.0000101617,0.0000014496,0.0010008812,6.8103277505
1160499,1544583809223.0,0.0000098074,0.0000150533,0.0000011913,0.0010018349,6.8103277505
1160500,1544583809224.0,0.0000097422,0.0000190363,0.0000007360,0.0010437965,6.8103277505
1160501,1544583809225.0,0.0000072155,0.0000222846,0.0000002926,0.0009741783,6.8103277505
1160502,1544583809226.0,0.0000026878,0.0000256788,-0.0000001611,0.0009784698,6.8103277505
1160503,1544583809227.0,-0.0000001810,0.0000284850,-0.0000007098,0.0010442734,6.8103277505
1160504,1544583809228.0,-0.0000012645,0.0000290127,-0.0000008862,0.0010001659,6.8103277505
1160505,1544583809229.0,-0.0000033514,0.0000267671,-0.0000017313,0.0009527206,6.8103277505
1160506,1544583809230.0,-0.0000032651,0.0000217959,-0.0000022545,0.0010020733,6.8103277505
1160507,1544583809231.0,-0.0000038793,0.0000165550,-0.0000024307,0.0010333061,6.8103277505


In [33]:
mergedf_fog_encoder_left.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                      False
linear_velocity(m/s)    False
dtype: bool

In [34]:
# inner

In [35]:
mergedf_fog_encoder_inner.tail(20)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,linear_velocity(m/s)
116130,1544583809037.0,-0.0000363645,-0.0000023723,-0.0000062061,0.0010011196,6.8655671486
116131,1544583809047.0,-0.0000432053,0.0000079868,-0.0000063139,0.0009989738,6.9159841399
116132,1544583809057.0,-0.0000578166,-0.0000081182,-0.0000065380,0.0010139942,6.8815550959
116133,1544583809067.0,-0.0000493888,-0.0000095094,-0.0000037340,0.0010478497,6.8162653171
116134,1544583809077.0,-0.0000349662,-0.0000104746,-0.0000003233,0.0009844303,6.9431653640
116135,1544583809087.0,-0.0000185079,0.0000026595,-0.0000030868,0.0010495186,6.8857638478
116136,1544583809097.0,-0.0000262275,0.0000013973,-0.0000048669,0.0010471344,6.8428111164
116137,1544583809107.0,-0.0000264319,0.0000017194,-0.0000012375,0.0010666847,6.8734584663
116138,1544583809117.0,-0.0000197881,-0.0000067770,-0.0000009368,0.0009648800,6.8787819108
116139,1544583809127.0,-0.0000107404,-0.0000081260,-0.0000050130,0.0009551048,6.8627785754


In [36]:
mergedf_fog_encoder_inner.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                      False
linear_velocity(m/s)    False
dtype: bool

# Output to .csv

In [37]:
mergedf_fog_encoder_inner.to_csv("sync_fog_encoder_inner.csv",index=False)

In [38]:
mergedf_fog_encoder_left.to_csv("sync_fog_encoder_left.csv",index=False)